# Minimizing the relative entropy

This example will determine the interaction parameters of a hard-sphere-like mixture that interacts through the Weeks-Chandler-Andersen (WCA) potential:

$$
    u_{ij}(r) = 4 \varepsilon \left[ \left(\frac{\sigma_{ij}}{r} \right)^{12} - \left(\frac{\sigma_{ij}}{r}\right)^6 + \frac{1}{4} \right], \quad r \le 2^{1/6} \sigma_{ij}.
$$

The WCA potential is the purely repulsive half of the Lennard-Jones potential, truncated and shifted at its minimum. Here, we will fix $\varepsilon = 1$ as the energy for all interactions, while we will try to adjust the size parameters $\sigma_{ij}$ between two particles of types $i$ and $j$. There are 3 parameters $\sigma_{11}$, $\sigma_{12}$, and $\sigma_{22}$.

Start off by importing `relentless`!

In [1]:
import relentless

## Model

We will work with a mixture that has two components 1 & 2 in the "dilute" limit of statistical mechanics, where the pair correlation function,

$$
    g_{ij}(r) = e^{-\beta u_{ij}(r)},
$$

is known exactly. Here, $\beta = 1/(k_{\rm B} T)$, $k_{\rm B}$ is the Boltzmann constant, and $T$ is the temperature.

### Target ensemble

Let's define a `target` thermodynamic state containing 10 particles of component 1 ($N_1 = 10$) and 5 particles of component 2 ($N_2 = 5$) in a cubic box with edge length $L = 10$. We will also set the nominal temperature as $T = 1.0$.

In [2]:
target = relentless.model.Ensemble(
    T=1.0, V=relentless.model.extent.Cube(L=10.0), N={"1": 10, "2": 5}
)

 The goal of relative entropy minimization is to determine interaction parameters that best reproduce a target ensemble, specifically the molecular structures. These are specified using the pair correlation functions. We have precomputed the $g_{11}$, $g_{12}$, and $g_{22}$ we are targeting for the mixture, and we have saved them to files `rdf_11.dat`, `rdf_12.dat`, and `rdf_22.dat`. You can download them [here](https://github.com/mphowardlab/relentless/tree/main/doc/source/guide/examples/binary_hard_spheres)! The first column are the values of $r$, and the second column are the values of $g_{ij}$. We load these and specify them as the `RDF` (radial distribution function) for the `target` ensemble:

In [3]:
for i, j in target.pairs:
    rdf = relentless.mpi.world.loadtxt(f"rdf_{i}{j}.dat")
    target.rdf[i, j] = relentless.model.ensemble.RDF(rdf[:, 0], rdf[:, 1])

Note that here we are using the `pairs` of types in the ensemble to programmatically loop through the files and set all of the RDFs. You could also do this manually if you like (e.g., `target.rdf["1", "1"]`), but that could get a bit tedious!

We also note that we are using the `loadtxt` method supplied by `relentless` for compatibility with MPI. There is a wrapper around `numpy.loadtxt` and there is very little overhead to using this even if you aren't using MPI support, so we recommend you get in the habit of using it in case you want to port your script to a supercomputer.

### Design variables

Next we are going to define the variables of our model that we want to parametrize. These are called `IndependentVariables` in `relentless`. They must start from an initial value, and they can have physical bounds.

We know that for two hard spheres having diameters $d_i$ and $d_j$, we typically choose $\sigma_{ij} = (d_i + d_j)/2$ to prevent overlap in the WCA potential. (Note that the WCA potential does not generate a true hard sphere, but this is actually better for relative entropy optimization because the WCA potential is differentiable.)

Hence, our model really has two parameters $d_1$ and $d_2$, then we can specify $\sigma_{11} = d_1$, $\sigma_{22} = d_2$, and $\sigma_{12} = (d_1+d_2)/2$. We make two design variables $d_1$ and $d_2$ and constrain them to be nonnegative and less than 5. We start them at slightly different values (this is just a guess!):

In [4]:
d = {
    "1": relentless.model.variable.IndependentVariable(
        value=1.5, name="d_1", low=0, high=5),
    "2": relentless.model.variable.IndependentVariable(
        value=2.5, name="d_2", low=0, high=5)
}

Then, we use these variables to parametrize the WCA potential. To do this, we create a `LennardJones` potential, which has the same functional form, with 2 types then specify self and cross pair coefficients according to the WCA form:

In [5]:
wca = relentless.model.potential.LennardJones(types=("1", "2"))
# "self" interactions
wca.coeff["1", "1"].update(
    {
        "epsilon": 1.0,
        "sigma": d["1"],
        "rmax": 2.**(1./6.) * d["1"],
        "shift": True,
    }
)
wca.coeff["2", "2"].update(
    {
        "epsilon": 1.0,
        "sigma": d["2"],
        "rmax": 2.**(1./6.) * d["2"],
        "shift": True,
    }
)

# "cross" interaction
sigma_12 = (d["1"]+d["2"])/2
wca.coeff["1", "2"].update(
    {
        "epsilon": 1.0,
        "sigma": sigma_12,
        "rmax": 2.**(1./6.) * sigma_12,
        "shift": True,
    }
)

Note that the pair coefficient matrix is symmetric, so we only have to specify the `("1", "2")` pair and not also the `("2", "1")` pair. In the process of doing this, we also performed basic arithmetic on `d_1` and `d_2` to get `sigma_12` and set all of the `rmax` parameters. These are called `DependentVariables` in relentless, and they will be automatically handled by the code for simple operations.

## Simulation

Now that the model is specified, we can run a simulation. As in previous examples, we first set up the simulation operations:

1. Initialize the particles randomly.
2. Run a short equilibration period with Langevin dynamics.
3. Run a production period with Langevin dynamics. We attach an ensemble analyzer that will be used in the relative entropy minimization later. The thermodynamic properties and RDF will be computed at regular intervals, and these intervals should be chosen based on what would be appropriate for sampling in a simulation.

In [6]:
init = relentless.simulate.InitializeRandomly(
    seed=42,
    T=target.T,
    V=target.V,
    N=target.N,
    diameters=d,
)

eq = relentless.simulate.RunLangevinDynamics(
    steps=2e4,
    timestep=0.005,
    T=target.T,
    friction=0.1,
    seed=2,
)

ens_avg = relentless.simulate.EnsembleAverage(
    check_thermo_every=20, check_rdf_every=2e3, rdf_dr=0.05
)
prod = relentless.simulate.RunLangevinDynamics(
    steps=2e5,
    timestep=0.005,
    T=target.T,
    friction=0.1,
    seed=3,
    analyzers=ens_avg,
)

We will use the `Dilute` simulator to run these operations, which does not really run most of them since it can straightforwardly compute its own ensemble. Hence, it can be used as a quick test of a script or to determine optimization parameters.

In [7]:
sim = relentless.simulate.Dilute(init, operations=[eq, prod])

We also create the potential tabulator for the simulation. We make sure that the stopping point is large enough to accommodate the largest possible cutoff that the WCA potential can adopt.

In [8]:
pot = relentless.simulate.Potentials([wca])
pot.pair.start = 1e-6
pot.pair.stop = 2.**(1./6.) * 5.
pot.pair.num = 1000
pot.pair.neighbor_buffer = 0.5

## Relative entropy optimization

Now that the model and simulation are specified, we are ready to optimize the relative entropy! First, we create the relative entropy objective function. It takes four arguments: the target ensemble, the simulation, the potentials to simulate, and the analysis operation that returns an ensemble average:

In [9]:
s_rel = relentless.optimize.RelativeEntropy(target, sim, pot, ens_avg)

Now we can optimize the relative entropy with respect to the parameters of our model. We will use standard steepest descent here, and we will stop the optimization once the norm of the gradient is less than a threshold. This type of optimization is not necessarily our first choice (we often prefer the fixed-step size variant of steepest descent with line search!), but it is simple to setup for illustrative purposes.

In [10]:
vars = [d["1"], d["2"]]
optimizer = relentless.optimize.FixedStepDescent(
    stop=relentless.optimize.GradientTest(1e-4, vars),
    max_iter=100,
    step_size=1e-2,
)
converged = optimizer.optimize(s_rel, vars)
print(f"Converged? {converged}")
print("d_1 = {:.1f}, d_2 = {:.1f}".format(d["1"].value, d["2"].value))

Converged? True
d_1 = 1.0, d_2 = 3.0


Depending on your problem, you may need to play around with the various parameters until you get a satisfactory answer. Here, the output of `optimize` shows that the process worked. The optimized values of the design variables are $d_1 = 1.0$ and $d_2 = 3.0$. These match the ones we used to create the RDFs!